In [ ]:
" / * chose the main atributies from our bibtex file*/ "

#!pip install bibtexparser

import bibtexparser
import json

# Read the .bib file
with open('raw/extra.bib', 'r') as bib_file:
    bib_database = bibtexparser.load(bib_file)

# Extract the required fields
articles = []
for entry in bib_database.entries:
    if entry['ENTRYTYPE'] == 'article':
        article = {
            'ID': entry['ID'],
            'title': entry.get('title', ''),
            'abstract': entry.get('abstract', ''),
            'journal': entry.get('journaltitle', ''),
            'author': entry.get('author', ''),
            'keywords': entry.get('keywords', '')
        }
        articles.append(article)

# Write to .json file
with open('raw/extra.json', 'w') as json_file:
    json.dump(articles, json_file, indent=4)

In [ ]:
" / * generate a list of keywords to make a cluster of research direction*/ "

import json
from openai import OpenAI


client = OpenAI(api_key="xxxx", base_url="https://api.deepseek.com")

with open('raw/extra_mini.json', 'r') as json_file:
    articles = json.load(json_file)

def generate_keywords(abstract):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a strategy management professor "},
            {"role": "user", "content": f"Generate 5 keywords for the following abstract:\n\n{abstract}\n\nKeywords:"},
        ],
        stream=False
    )
    keywords_text = response.choices[0].message.content
    keywords = [keyword.strip() for keyword in keywords_text.split('\n') if keyword.strip()]
    return keywords

def generate_main_purpose(abstract):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a strategy management professor"},
            {"role": "user", "content": f"Summarize the main purpose of the following research in one sentence, removing extra information:\n\n{abstract}\n\nMain Purpose:"},
        ],
        stream=False
    )
    main_purpose = response.choices[0].message.content.strip()
    return main_purpose


for article in articles:
    abstract = article.get('abstract', '')
    if abstract:
        keywords = generate_keywords(abstract)
        main_purpose = generate_main_purpose(abstract)
        article['keywords'] = keywords
        article['main_purpose'] = main_purpose

# Write the updated articles back to the JSON file
with open('raw/extra.json', 'w') as json_file:
    json.dump(articles, json_file, indent=4)